In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


# Define o nome do diretório
directory_name = 'datas'

# Obtém o caminho absoluto para o diretório atual
current_directory = os.getcwd()
datas_directory = os.path.join(current_directory, directory_name)

# Verifica se o diretório já existe
if not os.path.exists(datas_directory):
    # Se não existir, cria o diretório
    os.makedirs(datas_directory)

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root= datas_directory,
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root= datas_directory,
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle= False)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
    
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#Creating Models
#To define a neural network in PyTorch, we create a class that inherits from nn.Module. 
#We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function.
#To accelerate operations in the neural network, we move it to the GPU or MPS if available.

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

#Optimizing the Model Parameters
#To train a model, we need a loss function and an optimizer.

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

#In a single training loop, the model makes predictions on the training dataset
#(fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
#We also check the model’s performance against the test dataset to ensure it is learning.
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
#The training process is conducted over several iterations (epochs). 
#During each epoch, the model learns parameters to make better predictions. 
#We print the model’s accuracy and loss at each epoch; 
#we’d like to see the accuracy increase and the loss decrease with every epoch.

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

# Define um mapeamento de rótulos de classe para nomes de classes
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Função para exibir uma grade de imagens
def show_images(images, labels):
    
    plt.figure(figsize=(10, 10))
    for i in range(min(images.shape[0], 25)):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        # Remova a dimensão do canal extra
        plt.imshow(images[i].squeeze(), cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])
    plt.show()
    
    
    
    """
    plt.figure(figsize=(10, 10))
    for i in range(images.shape[0]):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        # Remova a dimensão do canal extra
        plt.imshow(images[i].squeeze(), cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])
    
    
   
    for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[labels[i]])
    """  
    
    

# Obtém um lote de imagens e rótulos do conjunto de dados de treinamento
images, labels = next(iter(train_dataloader))

# Exibe as imagens
show_images(images.numpy(), labels.numpy())

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using cpu device
Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.304948  [   64/60000]
loss: 2.286956  [ 6464/60000]
loss: 2.268370  [12864/60000]
loss: 2.248723  [19264/60000]
loss: 2.242040  [25664/60000]
loss: 2.219443  [32064/60000]
loss: 2.200244  [38464/60000]
loss: 2.189612  [44864/60000]
loss: 2.150996  [51264/60000]
loss: 2.150527  [57664/60000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 2.131864 

Done!
